Maersk

In [134]:
%pip install ydata-profiling pandas ipython plotly
import pandas as pd
from IPython.display import display, HTML

Note: you may need to restart the kernel to use updated packages.


# Introduction

A.P. Moller have committed to significantly reducing their carbon footprint, aiming to reach net-zero by 2040.

Greenhouse gas emissions can be split into 3 Scopes, as defined by the GHG Protocol (https://ghgprotocol.org/standards-guidance);
- Scope 1 - Greenhouse gases directly emitted by facilities you own (e.g., burning gas to power your car or office)
- Scope 2 - Emissions from electricity you purchase
- Scope 3 - Everything else: carbon emitted upstream (by your suppliers) and downstream (by your customers) (see https://ghgprotocol.org/scope-3-calculation-guidance-2)

Cloud Computing is comes under Scope 3, of which Maersk aim to see a 22% reduction in scope 3 by 2030, and 90% by 2040.

Maersk have adopted a cloud first (https://www.maersk.com/news/articles/2023/06/07/collaborating-to-take-logistics-to-the-next-level) approach to technology, and aim for this year to "target significant progress in the implementation of best-in-class technology platforms" (https://investor.maersk.com/static-files/31bf05a1-6f0c-4fbd-a3c7-3f58e044f668 - p22) meaning the investment in Cloud infrastructure which is already significant, will only increase.

# Aims of this Analysis

Cloud computing requires significant energy, with datacentres estimates to account for more than 14% of all global GHG emissions (https://www.datadynamicsinc.com/quick-bytes-cloud-computing-taking-the-energy-industry-by-storm-in-2023/).

By looking at the power usage effectiveness & carbon free energy percentage of cloud computing providers, across regions, recommendations could be made as to what cloud provider is the most suitable to use from an environmental perspective, and what region.

# Overview of Dataset

Cloud providers disclose regional data annually, 6 months after year-end.
This may include Power and Water Usage Effectiveness, carbon-free energy percentage, & location data for each region.

The Green Software Foundation project collates the data from the main providers (Amazon, GCP, Google) into a single source.  The data from 2019-2023 is based on actual figures, however 2024-2025 is projected data as the real data hasn't been fully collated yet.




# Import Data and Validate

## Import Data

In [135]:
'''Data Exports are read in from source CSV files'''

dataset = dict()

'''First part of dataset - data from GSF from 2019-2023 - actual data'''
dataset["gsf_data_from_2019"] = pd.read_csv('https://raw.githubusercontent.com/Green-Software-Foundation/real-time-cloud/refs/tags/v1.0/Cloud_Region_Metadata.csv')

'''Second part of dataset - data from GSF from 2024-2025 - some projected and estimated data'''
dataset["gsf_data_from_2024"] = pd.read_csv('https://raw.githubusercontent.com/Green-Software-Foundation/real-time-cloud/refs/tags/v1.0/Cloud_Region_Metadata_estimate.csv')

'''Not using this yet, but may use it later to combine with the others, to draw further insights'''
yearly_energy_dataset = pd.read_csv('https://storage.googleapis.com/emb-prod-bkt-publicdata/public-downloads/yearly_full_release_long_format.csv')

'''Check the two GSF datasets have the same columns'''
#(https://www.geeksforgeeks.org/python/python-difference-two-lists/)

'Check the two GSF datasets have the same columns'

## Validate the Data

### Visual Check

As a quick visual check of the data, we get a sample selection of 10 random rows.  This will help to confirm any immediate issues

In [136]:
'''Check the first dataset part'''
dataset["gsf_data_from_2019"].sample(10)

,year,cloud-provider,cloud-region,cfe-region,em-zone-id,wt-region-id,location,geolocation,provider-cfe-hourly,provider-cfe-annual,...,grid-carbon-intensity-average-consumption-annual,grid-carbon-intensity-marginal-consumption-annual,grid-carbon-intensity-average-production-annual,grid-carbon-intensity,total-ICT-energy-consumption-annual,total-water-input,renewable-energy-consumption,renewable-energy-consumption-goe,renewable-energy-consumption-ppa,renewable-energy-consumption-onsite
104,2022,Google Cloud,asia-south1,Maharashtra,IN-WE,IND,Mumbai,"19.0761,72.8775",0.24,NaN,...,747.67,679.409743,760.68,679.0,NaN,NaN,NaN,NaN,NaN,NaN
80,2023,Microsoft Azure,greececentral,Greece,GR,GR,Greece Central,"37.987, 23.745",NaN,NaN,...,368.75,NaN,342.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,2023,Google Cloud,northamerica-northeast2,Ontario,CA-ON,IESO_NORTH,Toronto,"43.7417,-79.3733",0.87,NaN,...,76.42,NaN,75.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,2022,Amazon Web Services,eu-west-3,France,FR,FR,Europe (Paris),"48.8567,2.3522",NaN,1.00,...,91.23,383.739084,72.00,384.0,NaN,NaN,NaN,NaN,NaN,NaN
156,2022,Amazon Web Services,eu-south-2,Spain,ES,ES,Europe (Spain),"40.3333,-3.8667",NaN,1.00,...,220.63,370.288102,220.31,370.0,NaN,NaN,NaN,NaN,NaN,NaN
33,2023,Google Cloud,us-east5,PJM,US-MIDA-PJM,PJM_SOUTHWEST_OH,Columbus,"41.4366,-97.3565",0.52,NaN,...,396.25,NaN,396.77,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,2022,Amazon Web Services,us-west-2,BPA,US-NW-BPAT,BPA,US West (Oregon),"45.5371,-122.65",NaN,1.00,...,86.61,427.188629,58.50,427.0,NaN,NaN,NaN,NaN,NaN,NaN
68,2023,Amazon Web Services,us-gov-west-1,BPA,US-NW-BPAT,BPA,GovCloud (US West),"45.5371,-122.65",NaN,NaN,...,119.76,NaN,68.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,2022,Microsoft Azure,westus,BPA,US-NW-BPAT,BPA,Washington: West US,"47.247,-119.82",NaN,0.47,...,86.61,427.188629,58.50,427.0,NaN,NaN,NaN,NaN,NaN,NaN
126,2022,Google Cloud,us-central1,MISO,US-MIDW-MISO,MISO_MASON_CITY,Iowa,"41.5725,-93.6105",0.92,NaN,...,526.14,539.251027,546.23,539.0,NaN,NaN,NaN,NaN,NaN,NaN


In [137]:
'''check the second (estimated) dataset part'''
dataset["gsf_data_from_2024"].sample(10)

,year,cloud-provider,cloud-region,cfe-region,em-zone-id,wt-region-id,location,geolocation,provider-cfe-hourly,provider-cfe-annual,...,grid-carbon-intensity-marginal-consumption-annual,grid-carbon-intensity-average-production-annual,grid-carbon-intensity,total-ICT-energy-consumption-annual,total-water-input,renewable-energy-consumption,renewable-energy-consumption-goe,renewable-energy-consumption-ppa,Unnamed: 23,renewable-energy-consumption-onsite
70,2025,Google Cloud,us-west4,NVE,US-NW-NEVP,NEVP,Las Vegas,"35.6011,-105.2206",0.292,NaN,...,456.753492,NaN,457.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,2024,Google Cloud,europe-central2,Poland,PL,PL,Warsaw,"52.23,21.0111",0.413,NaN,...,852.394151,NaN,852.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,2024,Google Cloud,us-central1,MISO,US-MIDW-MISO,MISO_MASON_CITY,Iowa,"41.5725,-93.6105",0.992,NaN,...,539.251027,NaN,539.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,2025,Google Cloud,europe-west1,Belgium,BE,BE,Belgium,"51.1333,4.5667",0.855,NaN,...,411.152318,NaN,411.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2024,Amazon Web Services,eu-south-2,Spain,ES,ES,Europe (Spain),"40.3333,-3.8667",NaN,1.0,...,370.288102,NaN,370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,2025,Google Cloud,us-south1,ERCOT,US-TEX-ERCO,ERCOT_NORTHCENTRAL,Dallas,"44.9221,-123.313",0.985,NaN,...,441.011686,NaN,441.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2025,Google Cloud,northamerica-northeast1,Quebec,CA-QC,HQ,Montréal,"45.5089,-73.5617",1.000,NaN,...,462.245178,NaN,462.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,2024,Microsoft Azure,centralus,MISO,US-MIDW-MISO,MISO,Iowa: Central US,"41.573,-93.608",NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,2025,Google Cloud,asia-northeast3,South Korea,KR,KOR,Seoul,"37.56,126.99",0.438,NaN,...,686.013604,NaN,686.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2024,Amazon Web Services,ap-east-1,Hong Kong,HK,HK,Asia Pacific (Hong Kong),"22.3,114.2",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Check Columns

As a visual check isn't that accurate, we wish to programmatically compare the two datasets columns to identify any disparity between the two.  The following function gets the column names as a list from both datasets and compares them to see which columns are not in both exports.

The results are returned in dictionary format along with summary totals, to make it easier to analyse

In [138]:
def return_unique_entries_between_two_lists(list1, list2):
    """Take two lists and return unique entries between them"""
    '''We cast the lists into sets as they can be subtracted from each other, unlike
       lists, and then we subtract list2 from list 1 to get the unique entries from
       list1'''
    unique_to_list_1 = list(set(list1) - set(list2))
    '''Then we flip the order, subtracting list1 from list2, to get the unique entries
       from list2'''
    unique_to_list_2 = list(set(list2) - set(list1))
    return unique_to_list_1, unique_to_list_2

def column_differences(dataset_1, dataset_2, dataset_title_1, dataset_title_2):
    """List the unique columns from each dataset and summarise findings"""
    unique_col_1, unique_col_2 = return_unique_entries_between_two_lists(dataset_1.columns, dataset_2.columns)

    return {
        dataset_title_1 : {
            "unique_columns_list" : ",".join(unique_col_1),
            "total_unique_columns" : len(unique_col_1),
            "total_columns" : len(dataset_1.columns),
        },
        dataset_title_2 : {
            "unique_columns_list" : ",".join(unique_col_2),
            "total_unique_columns" : len(unique_col_2),
            "total_columns" : len(dataset_2.columns),
        },
        "combined" : {
            "unique_columns_list" : ",".join(unique_col_1 + unique_col_2),
            "total_unique_columns" : len(unique_col_1 + unique_col_2),
            "total_columns" : len(set(dataset_1.columns).union(dataset_2.columns)),
        }
    }

column_difference_summary = column_differences(
    dataset_1=dataset["gsf_data_from_2019"],
    dataset_2=dataset["gsf_data_from_2024"],
    dataset_title_1="gsf_data_from_2019",
    dataset_title_2="gsf_data_from_2024")

def style_column_differences_datafrme(row):
    """"""
    colour = ''
    return_value = []
    '''Check the value of the final cell in the row and assign style based on that'''
    for _ in row:
        colour = 'white'
        if row.name == 'unique_columns_list':
            if _ == '':
                colour = 'green'
            else:
                colour = 'red'
        if row.name == 'total_unique_columns':
            if _ == 0:
                colour = 'green'
            else:
                colour = 'red'
        if row.name == 'total_columns':
            colour = 'white'
        return_value.append(f'color: {colour}')
    return return_value
'''Display a header for our summary table'''
display(HTML('<span style="color: white;font-style:bold"><br><h2>Unique columns between the two datasets</h2></span>'))
display(pd.DataFrame(column_difference_summary).fillna(0).style.apply(style_column_differences_datafrme,axis=1))

,gsf_data_from_2019,gsf_data_from_2024,combined
unique_columns_list,,Unnamed: 23,Unnamed: 23
total_unique_columns,0,1,1
total_columns,24,25,25


# Clean Data
## Drop Unique Columns

Merging two datasets will only work if the data is consistent across both sets.  There is only one unique column identified, which is entitled 'Unnamed: 23' - this is meaningless to us, so we can drop it and have a unified columns list.

In [139]:
dataset["gsf_data_from_2024"].drop(column_difference_summary["gsf_data_from_2024"]["unique_columns_list"], axis=1, inplace=True)
dataset["gsf_data_from_2024"].head()

,year,cloud-provider,cloud-region,cfe-region,em-zone-id,wt-region-id,location,geolocation,provider-cfe-hourly,provider-cfe-annual,...,grid-carbon-intensity-average-consumption-annual,grid-carbon-intensity-marginal-consumption-annual,grid-carbon-intensity-average-production-annual,grid-carbon-intensity,total-ICT-energy-consumption-annual,total-water-input,renewable-energy-consumption,renewable-energy-consumption-goe,renewable-energy-consumption-ppa,renewable-energy-consumption-onsite
0,2025,Amazon Web Services,af-south-1,South Africa,ZA,ZA,Africa (Cape Town),"-33.9253,18.4239",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025,Amazon Web Services,ap-east-1,Hong Kong,HK,HK,Asia Pacific (Hong Kong),"22.3,114.2",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025,Amazon Web Services,ap-northeast-1,Tokyo,JP-TK,JP_TK,Asia Pacific (Tokyo),"35.6897,139.692",NaN,NaN,...,NaN,535.953159,NaN,536.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2025,Amazon Web Services,ap-northeast-2,South Korea,KR,KOR,Asia Pacific (Seoul),"37.56,126.99",NaN,NaN,...,NaN,686.013604,NaN,686.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2025,Amazon Web Services,ap-northeast-3,Kansai,JP-KN,JP_KN,Asia Pacific (Osaka),"34.6939,135.502",NaN,NaN,...,NaN,550.532231,NaN,551.0,NaN,NaN,NaN,NaN,NaN,NaN


## Confirm columns match

After dropping the unique columns we run a final check to ensure the columns match across both datasets

In [140]:
column_difference_summary = column_differences(
    dataset_1=dataset["gsf_data_from_2019"],
    dataset_2=dataset["gsf_data_from_2024"],
    dataset_title_1="gsf_data_from_2019",
    dataset_title_2="gsf_data_from_2024")

'''Display a header for our summary table'''
display(HTML('<span style="color: white;font-style:bold"><br><h2>Unique columns between the two datasets</h2></span>'))
display(pd.DataFrame(column_difference_summary).fillna(0).style.apply(style_column_differences_datafrme,axis=1))


,gsf_data_from_2019,gsf_data_from_2024,combined
unique_columns_list,,,
total_unique_columns,0,0,0
total_columns,24,24,24


## Check for null columns

There may be columns without any data entered in at all, which can be identified by having 'NaN' as every value.  If this is true across both datasets we are going to combine then we can drop them.   We can use the following function to identify them.



In [141]:
def check_for_null_columns(dataframe_to_check):
    """This function returns a list of columns that only have null values"""
    '''Find out how many entries there are in the dataset'''
    total_rows = dataframe_to_check.shape[0]
    '''count up how many fields under each column are NaN/null'''
    df_to_count_na_fields = dataframe_to_check.isna().sum(axis=0)
    '''The NaN totals were returned in a pandas Series.  Lets rename its only column
       so it is something more meaningful than <unnamed>'''
    df_to_count_na_fields.rename("total", inplace=True)
    '''Return only rows that have a total number of NaN the same as the total number of rows'''
    na_rows = df_to_count_na_fields[df_to_count_na_fields == total_rows]
    '''return the columns with only NaN values as a list'''
    return na_rows.index.tolist()

## Find Columns with NaN/null fields in both datasets
Now we use that function to list the columns that are empty across both datasets.
We only temporarily combine them, as we may need to do some more work on them independently before we are ready to properly merge


In [142]:
'''List null columns that are common across both datasets'''
common_empty_columns = check_for_null_columns(
    pd.concat(
        [dataset["gsf_data_from_2019"],
         dataset["gsf_data_from_2024"]],
        ignore_index=True)
    )
common_empty_columns

['total-ICT-energy-consumption-annual',
 'total-water-input',
 'renewable-energy-consumption',
 'renewable-energy-consumption-goe',
 'renewable-energy-consumption-ppa',
 'renewable-energy-consumption-onsite']

## Drop common NaN columns from both datasets
Loop through the list of Nan columns and drop them from both datasets.  If we were ready to combine these, we could just use 'dropna' but doing so now would risk dropping a column from one dataset that has data in another, and having a mismatch of columns.


In [143]:
'''Drop all columns with NaN as the only value'''

for column in common_empty_columns:
    dataset['gsf_data_from_2019'].drop(column, axis=1, inplace=True)
    dataset['gsf_data_from_2024'].drop(column, axis=1, inplace=True)


'''Check the column listing now'''
column_differences(
    dataset_1=dataset["gsf_data_from_2019"],
    dataset_2=dataset["gsf_data_from_2024"],
    dataset_title_1="gsf_data_from_2019",
    dataset_title_2="gsf_data_from_2024")

{'gsf_data_from_2019': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 18},
 'gsf_data_from_2024': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 18},
 'combined': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 18}}

## Identify columns with NaN/null in only one dataset
If there is incomplete data in only one dataset, we should identify it and see if we can take some action on it

In [144]:
def map_list_to_dict(list1, list2, value1, value2):
    """Take two lists and convert them into a dictionary, with
       each list member being a key in the dictionary, and the
       value being a common one associated with all members of
       that list.  The two lists will have unique values so there
       are no duplicate keys.  The purpose of this function is to make
       it much easier to identify which dataset the identified column
       belongs to"""
    dictionary_to_return = {}
    for list_member in list1:
        dictionary_to_return[list_member] = value1
    for list_member in list2:
        dictionary_to_return[list_member] = value2
    return dictionary_to_return


def compare_two_dataset_and_return_unique_null_columns(dataset_1, dataset_2, dataset_title_1, dataset_title_2):
    """docstring will go here eventually when i get to it"""
    '''Get list of null columns from first dataset'''
    null_columns = dict()
    null_columns[dataset_title_1] = check_for_null_columns(dataset_1)
    '''Get list of null columns from second dataset'''
    null_columns[dataset_title_2] = check_for_null_columns(dataset_2)
    '''Identify columns that are only null in one of the datasets'''
    unique_null_col_1, unique_null_col_2 = return_unique_entries_between_two_lists(null_columns[dataset_title_1], null_columns[dataset_title_2])
    '''Create a dictionary to hold the list of columns that have null values in only one dataset
       and the name of the dataset'''
    return map_list_to_dict(unique_null_col_1, unique_null_col_2, dataset_title_1, dataset_title_2)


columns_null_in_only_one_dataset = compare_two_dataset_and_return_unique_null_columns(dataset["gsf_data_from_2019"],
                                                                                      dataset["gsf_data_from_2024"],
                                                                                      "gsf_data_from_2019",
                                                                                      "gsf_data_from_2024")


'''Display columns that have null values in only one dataset
   and the name of the dataset'''
columns_null_in_only_one_dataset

{'grid-carbon-intensity-average-production-annual': 'gsf_data_from_2024'}

## Drop unique null columns as not required
The list of columns with null data in only one dataset reveals only one column = 'grid-carbon-intensity-average-production-annual'.   Looking at the definition of this from https://www.electricitymaps.com/content/consumption-carbon-intensity, we can see that this is a measurement of only the energy produced locally in a region, discounting any used that has been imported.   The consumption measurements are more accurate for our purposes as they show all the energy available in a region regardless of where it's from.

We can conclude then, that the column isn't required and can be dropped



In [145]:
for title in ["gsf_data_from_2019","gsf_data_from_2024"]:
    dataset[title].drop('grid-carbon-intensity-average-production-annual',axis=1, inplace=True)

column_differences(
    dataset_1=dataset["gsf_data_from_2019"],
    dataset_2=dataset["gsf_data_from_2024"],
    dataset_title_1="gsf_data_from_2019",
    dataset_title_2="gsf_data_from_2024")

{'gsf_data_from_2019': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 17},
 'gsf_data_from_2024': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 17},
 'combined': {'unique_columns_list': '',
  'total_unique_columns': 0,
  'total_columns': 17}}

## Drop rows with null/NaN values in the columns we are interested in

The columns we are interested in analysing to start with are :
   - power-usage-effectiveness: PUE ratio for the region (1.0 = 100% of energy goes to IT, 1.0-1.5 = v efficient, 2.0+ = inefficient)
   - provider-cfe-annual (0 = 0% carbon free energy & 100% fossil fuels, 1.0 = 100% cfe & 0% fossil fuels)
   - provider-cfe-hourly: same as above, but meant to represent cfe proportion by hourly usage.

The two provider columns do seem to represent the same thing, and looking at sample rows of the datasets we can see that neither column is comprehensively or consistently filled out.  To conclude which data column to use, we should compare the two column values and see where the gaps are, and confirm that where we have both sets of information, they do resemble each other.


In [146]:
def debug_display(display_object, display_enabled=True):
    """A little helper function for me to use in place of display() so i can control whether i output all details or not,
       to avoid flooding the Jupyter notebook output cell with too much info"""
    if display_enabled:
        display(display_object)


def cast_float_to_int_and_set_nan_to_zero(input_dataframe):
    '''Set all NaN to 0'''
    input_dataframe.fillna(0, inplace=True)
    '''Cast all floats to integer as merging tables seems to set the integers to floats'''
    input_dataframe[input_dataframe.select_dtypes(include='float').columns] = input_dataframe.select_dtypes(include='float').astype(int)
    return input_dataframe

def compare_empty_values_across_two_columns(df, column_1, column_2, sort_by, debug_on=False):
    """Reports on NaN values across two columns in a dataframe.  Outputs a collated table that shows how many rows
       - have values in both columns
       - have have no value in either
       - have values in just one column but not the other
       - total rows that match the criteria.
       The list 'sort_by' is used to determine not only which columns to sort by, but also what to match on, when filtering
       for totals, and for merging each type of total into the collated table"""
    '''Total number of rows'''
    debug_display("Total number of rows", debug_on)
    total_row_counts_df=df[sort_by].value_counts().to_frame().sort_values(by=sort_by).rename(columns={'count':'total_rows'})
    debug_display(total_row_counts_df, debug_on)

    '''Number of rows where we have neither values filled in'''
    debug_display("Number of rows where we have neither values filled in", debug_on)
    both_empty_rows_df = df[df[column_1].isna() & df[column_2].isna()][sort_by].value_counts().to_frame().sort_values(by=sort_by).rename(columns={'count':'both_empty'})
    debug_display(both_empty_rows_df, debug_on)
    total_row_counts_df = pd.merge(total_row_counts_df,both_empty_rows_df, how='left', on=sort_by)

    f'''Rows where {column_1} is NaN and {column_2} is not'''
    debug_display(f"Number of entries where {column_1} is missing, but {column_2} is present", debug_on)
    column_1_df = df[df[column_1].isna() & df[column_2].notna()][sort_by].value_counts().to_frame().sort_values(by=sort_by).rename(columns={'count':f"only_{column_1}_empty"})
    debug_display(column_1_df, debug_on)
    total_row_counts_df = pd.merge(total_row_counts_df,column_1_df, how='left', on=sort_by)

    f'''Rows where {column_2} is NaN and {column_1} is not'''
    debug_display(f"Number of entries where {column_2} is missing, but {column_1} is present", debug_on)
    column_2_df = df[df[column_1].notna() & df[column_2].isna()][sort_by].value_counts().to_frame().sort_values(by=sort_by).rename(columns={'count':f"only_{column_2}_empty"})
    debug_display(column_2_df, debug_on)
    total_row_counts_df = pd.merge(total_row_counts_df,column_2_df, how='left', on=sort_by)

    '''Number of rows where we have both entries filled in'''
    debug_display("Number of rows where we have both entries filled in", debug_on)
    both_have_values_df = df[df[column_1].notna() & df[column_2].notna()][sort_by].value_counts().to_frame().sort_values(by=sort_by).rename(columns={'count':'both_have_values'})
    debug_display(both_have_values_df, debug_on)
    total_row_counts_df = pd.merge(total_row_counts_df,both_have_values_df, how='left', on=sort_by)

    '''clean up number columns'''
    total_row_counts_df = cast_float_to_int_and_set_nan_to_zero(total_row_counts_df)

    '''Add up totals columns using sum(), to give us a Series containing the sum total of each column,
       convert to a dataframe (.to_frame) and then transpose the axis (.T) so we get a row with headings
       that mostly align with our totals dataframe.  '''

    total_of_totals_df = total_row_counts_df.sum().to_frame().T
    '''create new index for our totals'''
    all_totals_index = pd.MultiIndex.from_tuples([('All', 'All')],
           names=['cloud-provider', 'dataset'])
    '''Set the index on our single-row totals dataframe'''
    total_of_totals_df.set_index(all_totals_index, inplace=True)

    '''We add the row of overall totals to the bottom of the totals dataframe, by concatenating the totals dataframe
       with our new, single row dataframe'''
    total_row_counts_df = pd.concat(
        [total_row_counts_df,
         total_of_totals_df],
         ignore_index=False)

    '''return total counts dataframe as we may need it later'''
    return total_row_counts_df

def check_dataframe_columns_match(input_dataframe, column_1, column_2):
    """Check if two columns in a dataframe have matching values in every row"""
    input_dataframe['matches'] = input_dataframe[column_1] == input_dataframe[column_2]

    display(HTML('<span style="color: grey;font-style:italic"><br>Checking that where we have rows with both entries filled in, those entries are the same value.....</span>'))
    display(input_dataframe)
    if input_dataframe['matches'].all():
        display(HTML(f'<span style="color: green;font-weight:bold"><br>The {column_1} and {column_2} column values DO match in every row where both column values are populated</span>'))
    else:
        display(HTML(f'<span style="color: red;font-weight:bold"><br>The {column_1} and {column_2} column values DO NOT all match</span>'))

'''Combine the two datasets into one, but add an additional column so we know which one they came from'''
'''Make a copy of the original dataset dataframes in case we need to refer to them again later, after
   we've cleaned the data'''
temp_df_a = dataset["gsf_data_from_2019"].copy()
temp_df_a['dataset']='2019-2023'
temp_df_b = dataset["gsf_data_from_2024"].copy()
temp_df_b['dataset']='2024-2025'

'''use shortened name for df to make dataframe filtering easier to read'''
df = pd.concat(
        [temp_df_a,
         temp_df_b],
        ignore_index=True)

column_a = "provider-cfe-hourly"
column_b = "provider-cfe-annual"
sort_by_columns=['cloud-provider','dataset']
cfe_totals_df = compare_empty_values_across_two_columns(df, column_a, column_b, sort_by=sort_by_columns)

'''Display comined totals, including overall totals of all columns'''
display(HTML(f'<span style="color: grey;font-style:italic"><br>Combined totals of our comparisons between columns <b>{column_a}</b> and <b>{column_b}</b>, including sum totals of each column</span>'))
display(cfe_totals_df)

'''Filter data for only rows with both entries filled in'''
are_populated_columns_equal_df = df[df[column_a].notna() & df[column_b].notna()][['year','cloud-provider','cfe-region','provider-cfe-hourly','provider-cfe-annual']]

'''Display whether the two columns are equal or not, for the rows with both entries filled in'''
check_dataframe_columns_match(are_populated_columns_equal_df, column_a, column_b)


total_rows  both_empty  \
cloud-provider      dataset                             
Amazon Web Services 2019-2023          94          62   
                    2024-2025          64          26   
Google Cloud        2019-2023         160           0   
                    2024-2025          78           0   
Microsoft Azure     2019-2023          56          32   
                    2024-2025          56          10   
All                 All               508         130   

                               only_provider-cfe-hourly_empty  \
cloud-provider      dataset                                     
Amazon Web Services 2019-2023                              32   
                    2024-2025                              38   
Google Cloud        2019-2023                               0   
                    2024-2025                               0   
Microsoft Azure     2019-2023                              22   
                    2024-2025                              44   
All                 All                                   136   

                               only_provider-cfe-annual_empty  \
cloud-provider      dataset                                     
Amazon Web Services 2019-2023                               0   
                    2024-2025                               0   
Google Cloud        2019-2023                             160   
                    2024-2025                              78   
Microsoft Azure     2019-2023                               1   
                    2024-2025                               0   
All                 All                                   239   

                               both_have_values  
cloud-provider      dataset                      
Amazon Web Services 2019-2023                 0  
                    2024-2025                 0  
Google Cloud        2019-2023                 0  
                    2024-2025                 0  
Microsoft Azure     2019-2023                 1  
                    2024-2025                 2  
All                 All                       3

,year,cloud-provider,cfe-region,provider-cfe-hourly,provider-cfe-annual,matches
172,2022,Microsoft Azure,Sweden,1.0,1.0,True
403,2025,Microsoft Azure,Sweden,1.0,1.0,True
502,2024,Microsoft Azure,Sweden,1.0,1.0,True


### Based on the above, we can combine the two provider-cfe columns to make a more comprehensive dataset:

In [147]:
df['cfe-ratio'] = df['provider-cfe-hourly'].where(df['provider-cfe-hourly'].notnull(), df['provider-cfe-annual'])

### Once combined, check the totals to ensure the values in the two columns were combined successfully

In [148]:
'''First make a copy of the original totals dataframe, but only keep the columns showing total rows and how many rows had no value in both cfe columns'''
total_check_df = cfe_totals_df.copy()[['total_rows','both_empty']]
'''Now create a dataframe of just the counts of the rows with empty values in the new combined cfe-ratio column'''
na_df = df[df['cfe-ratio'].isna()][sort_by_columns].value_counts().to_frame().sort_values(by=sort_by_columns).rename(columns={'count':f"cfe-empty"})
'''Merge this new column onto the existing one'''
total_check_df = total_check_df.merge(na_df, how='left', on=sort_by_columns)

'''Add uo the number of empty cells in the cfe-ratio column, by totalling the cfe-empty totals in our summary table and then write it to All,All'''
total_check_df.loc[('All', 'All'), 'cfe-empty'] = total_check_df['cfe-empty'].sum()

'''clean up number columns'''
total_check_df = cast_float_to_int_and_set_nan_to_zero(total_check_df)

'''Validate the two empty columns match - this way we know we havent missed a row when we combined the data from our two columns'''
check_dataframe_columns_match(total_check_df, "both_empty", "cfe-empty")

total_rows  both_empty  cfe-empty  matches
cloud-provider      dataset                                              
Amazon Web Services 2019-2023          94          62         62     True
                    2024-2025          64          26         26     True
Google Cloud        2019-2023         160           0          0     True
                    2024-2025          78           0          0     True
Microsoft Azure     2019-2023          56          32         32     True
                    2024-2025          56          10         10     True
All                 All               508         130        130     True

### Now drop rows that have no cfe data from the dataset and confirm that there are no empty cells

In [149]:
df = df[df['cfe-ratio'].notna()]
new_totals_df = compare_empty_values_across_two_columns(df, 'cfe-ratio', 'dataset', sort_by=sort_by_columns)[['total_rows','both_have_values']].rename(columns={'both_have_values':'has_value'})
check_dataframe_columns_match(new_totals_df, "total_rows", "has_value")

total_rows  has_value  matches
cloud-provider      dataset                                  
Amazon Web Services 2019-2023          32         32     True
                    2024-2025          38         38     True
Google Cloud        2019-2023         160        160     True
                    2024-2025          78         78     True
Microsoft Azure     2019-2023          24         24     True
                    2024-2025          46         46     True
All                 All               378        378     True

### Now we have confirmed our combined CFE column has been collated successfully, we can drop the two source ones

In [150]:
for old_column in ['provider-cfe-hourly','provider-cfe-annual']:
    if old_column in df.columns:
        df.drop(columns=[old_column], inplace=True)

### Check all cloud-providers have data for all years in the dataset

In [151]:
def change_table_style_based_on_how_many_cells_are_true(row):
    """This function we will use to set the style of our dataframe based on how many True
        cells are in a row.   The last column is ignored when calculating what the style
        should be as i added this to show if the other columns matched - much like the
        style colour, but more binary being just true/false, rather than the traffic light
        colouring being applied here, using thresholds for each colour"""
    return_style = ''
    return_value = []
    total_booL_columns = 0
    total_booL_is_true = 0
    for _ in row[:-1]:
        if type(_) == bool:
            total_booL_columns=total_booL_columns+1
            if _:
                total_booL_is_true=total_booL_is_true+1

    pct_true = int((total_booL_is_true / total_booL_columns) * 100)
    if pct_true == 100:
        return_style = 'background-color: lightgreen; color: black'
    if 100 > pct_true >= 66:
        return_style = 'background-color: yellow; color: black'
    if 66 > pct_true >= 33:
        return_style = 'background-color: orange; color: black'
    if 33 > pct_true >= 0:
        return_style = 'background-color: red; color: black'
    for _ in row:
          return_value.append(return_style)
    return return_value

'''Create a list of the different cloud providers in the dataset'''
cloud_providers_in_ds = df['cloud-provider'].unique().tolist()
'''Create a list of the years included in the dataset'''
years_in_ds = df['year'].unique().tolist()

'''Loop through years and cloud providers to check if there is a valid combination of both in the dataset
    If there isn't then it means we are missing data for some years for that cloud provider'''
data_check = []
'''loop through years'''
for year in years_in_ds:
    '''For each year we start with our result being true, indicating that year has entries
       for all cloud providers'''
    year_result = True
    '''For each year we create a dictionary that includes that year as one of the kv pairs'''
    year_status = {'year' : year}
    '''loop through cloud providers'''
    for cloud_provider in cloud_providers_in_ds:
        '''If a cloud provider isnt found for that year we set the result to False so we
           know that years dataset is incomplete'''
        if len(df[(df['year'] == year) & (df['cloud-provider'] == cloud_provider)]) == 0:
            '''if our row count when filtering for the current year and provider is zero,
               then set the year_result to False (dataset for that year is incomplete) and
               set a kv pair for the cloud provider in our year dictionary to False'''
            year_status[cloud_provider] = False
            year_result=False
        else:
            '''If we do find rows, we can leave the year_result alone, as it started True,
               but we set a kv pair for the cloud provider in our year dictionary to True'''
            year_status[cloud_provider] = True
    '''Once we've looped through all cloud prpviders we set a kv pair for 'complete' to be
       either True/False depending on year_result.  This will be a summary column in this
       totals dataframe to show if the dataset for that year has all cloud-providers or
       is incomplete'''
    year_status['complete'] = year_result
    '''Add the dictionary for the current year to our list.  Once the list is complete, we
       can turn this list of dicts into a dataframe'''
    data_check.append(year_status)

'''Display a header for our summary table'''
display(HTML('<span style="color: white;font-style:bold"><br><h2>Cloud provider presence in dataset by year</h2></span>'))

'''turn the list of dicts into a dataframe'''
results_df = pd.DataFrame(data_check).sort_values(by=['year'])
'''apply some styling to the dataframe so it is colour coded based on how many columns have the value of
   True.  We use a custom function for this'''
results_with_style_df = results_df.style.apply(change_table_style_based_on_how_many_cells_are_true, axis=1)
'''Display our styled dataframe'''
display(results_with_style_df)
'''Display a legend for the dataframe'''
display(HTML(f'<span style="color: white;font-style:italic"><br>In the above table, the rows are colour-coded based on how many cloud providers have data that year;\
 <br>- years with all 3 cloud providers are highlighted in green; \
 <br>- years with only two are highlighted in yellow;\
 <br>- years with only 1 are orange;\
 <br>- years with none are red.</span>'))

'''Get a list of the years that contain data for all cloud providers, then display it'''
complete_years = results_df[results_df['complete'] == True]['year'].sort_values().tolist()
display(HTML(f'<span style="color: white;font-style:italic"><br>Complete years are : </span>'))
display(complete_years)




,year,Google Cloud,Microsoft Azure,Amazon Web Services,complete
4,2019,True,False,False,False
3,2020,True,False,False,False
2,2021,True,False,True,False
1,2022,True,True,True,True
0,2023,True,True,False,False
6,2024,True,True,True,True
5,2025,True,True,True,True


[2022, 2024, 2025]

### Filter for only the complete years

It will be interesting to see what difference keeping the partial year dataset will make, so lets take forward two versions of this: one with all the years, and one with only the years with data from all cloud-providers

In [152]:
'''Create a dataframe to represent the dataset with all years present, even those that don't contain
   data for all cloud providers'''
all_year_df = df.copy()
partial_year_df = df[df['year'].isin(complete_years)]

'''Validate that our partial dataframe only has the complete years in'''
validation_check = partial_year_df['year'].sort_values().unique().tolist() == complete_years
text_colour = 'green' if validation_check else 'red'
display(HTML(f'<span style="color: {text_colour};font-style:italic"><br>It is {validation_check} that the years in the partial table are the ones we expect </span>'))

## Filter for valid PUE Effectiveness on both datasets and compare

In [153]:
def change_table_style_based_on_pct_in_cell(row):
    """Custom style our dataframe so that the percentage decrease shown in the last column
       dictates the colour of the row.  100% is red (all rows gone), 0% is green (all rows remain)
       with orange, yellow and yellowgreen used for the steps inbetween"""
    colour = ''
    return_value = []
    '''Check the value of the final cell in the row and assign style based on that'''
    if row.iloc[-1] == 100: colour = 'red'
    if 100 > row.iloc[-1] >= 75: colour = 'orange'
    if 75 > row.iloc[-1] >= 50: colour = 'yellow'
    if 50 > row.iloc[-1] >= 1: colour = 'yellowgreen'
    if row.iloc[-1] == 0: colour = 'green'
    for _ in row:
          return_value.append(f'background-color: {colour}; color: black')
    return return_value

def show_row_count_change_when_dropping_empty_values_cells_from_specified_column(input_df,target_column,sort_columns):
    row_counts_df = input_df[sort_columns].value_counts().to_frame().rename(columns={'count':'before'})
    row_counts_post_drop_df = input_df.dropna(subset=[target_column])[sort_columns].value_counts().to_frame().rename(columns={'count':'after'})
    output_df = pd.merge(row_counts_df,row_counts_post_drop_df,how='left', on=sort_columns).fillna(0)
    output_df['% decrease'] = (100-((output_df['after'] / output_df['before']) * 100))
    output_df[['before','after']] = output_df[['before','after']].astype(int)
    return output_df.style.apply(change_table_style_based_on_pct_in_cell, axis=1).format("{:.1f}")

'''Lets ensure the PUE rows we have are all valid'''
display(HTML('<span style="color: white;font-style:bold"><br><h1>Number of rows, by year & provider : before and after cleansing rows with empty PUE values, with % decrease in rows</h1></span>'))
display(HTML('<span style="color: white;font-style:bold"><br><h2>All years (including years with partial provider data)</h2></span>'))
display(
    show_row_count_change_when_dropping_empty_values_cells_from_specified_column(
        all_year_df,
        "power-usage-effectiveness",
        ['cloud-provider','year'] ))


display(HTML('<span style="color: white;font-style:bold"><br><h2>Subset of years (including only those with data from all providers)</h2></span>'))
display(
    show_row_count_change_when_dropping_empty_values_cells_from_specified_column(
        partial_year_df,
        "power-usage-effectiveness",
        ['cloud-provider','year'] ))

display(HTML('<span style="color: white;font-style:italic"><br>In the above tables, the rows are colour-coded based on the percentage reduction in record counts after removing rows where PUE effectiveness is null;\
 <br>- 0% reduction is green: we have lost no rows \
 <br>- 1-49% reduction is yellow-green\
 <br>- 50-74% is yellow\
 <br>- 75-99% is orange\
 <br>- 100% is red : all record had no PUE value so have been removed</span>'))

# Data Analysis Phase
## Checkiing for correlation

In [154]:
import plotly.express as px

def generate_correlation_heatmap(input_df, heatmap_title):
    heatmap_data = input_df.select_dtypes(include='number').corr().fillna(0)
    display(HTML(f'<span style="color: white;font-style:bold"><br><h2>{heatmap_title}</h2></span>'))
    display(
        heatmap_data.style.background_gradient(cmap='coolwarm').format(precision=2))
    return heatmap_data


all_year_graph = generate_correlation_heatmap(all_year_df,"Heatmap of dataset containing all years (including years with partial provider data)")
partial_year_graph = generate_correlation_heatmap(partial_year_df,"Heatmap of dataset for subset of years (including only years with all providers)")





,year,power-usage-effectiveness,water-usage-effectiveness,provider-carbon-intensity-market-annual,provider-carbon-intensity-average-consumption-hourly,grid-carbon-intensity-average-consumption-annual,grid-carbon-intensity-marginal-consumption-annual,grid-carbon-intensity,cfe-ratio
year,1.00,0.01,-0.01,0.00,-0.13,-0.10,0.01,0.01,0.21
power-usage-effectiveness,0.01,1.00,0.28,0.00,0.17,0.28,0.30,0.30,0.06
water-usage-effectiveness,-0.01,0.28,1.00,0.00,0.00,0.22,-0.29,-0.29,-0.19
provider-carbon-intensity-market-annual,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
provider-carbon-intensity-average-consumption-hourly,-0.13,0.17,0.00,0.00,1.00,0.93,0.60,0.60,-0.72
grid-carbon-intensity-average-consumption-annual,-0.10,0.28,0.22,0.00,0.93,1.00,0.35,0.35,-0.55
grid-carbon-intensity-marginal-consumption-annual,0.01,0.30,-0.29,0.00,0.60,0.35,1.00,1.00,-0.14
grid-carbon-intensity,0.01,0.30,-0.29,0.00,0.60,0.35,1.00,1.00,-0.14
cfe-ratio,0.21,0.06,-0.19,0.00,-0.72,-0.55,-0.14,-0.14,1.00


,year,power-usage-effectiveness,water-usage-effectiveness,provider-carbon-intensity-market-annual,provider-carbon-intensity-average-consumption-hourly,grid-carbon-intensity-average-consumption-annual,grid-carbon-intensity-marginal-consumption-annual,grid-carbon-intensity,cfe-ratio
year,1.00,-0.02,-0.01,0.00,-0.04,0.07,0.01,0.01,0.02
power-usage-effectiveness,-0.02,1.00,0.28,0.00,0.14,0.31,0.30,0.30,0.07
water-usage-effectiveness,-0.01,0.28,1.00,0.00,0.00,0.22,-0.29,-0.29,-0.19
provider-carbon-intensity-market-annual,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
provider-carbon-intensity-average-consumption-hourly,-0.04,0.14,0.00,0.00,1.00,0.97,0.60,0.60,-0.79
grid-carbon-intensity-average-consumption-annual,0.07,0.31,0.22,0.00,0.97,1.00,0.35,0.35,-0.45
grid-carbon-intensity-marginal-consumption-annual,0.01,0.30,-0.29,0.00,0.60,0.35,1.00,1.00,-0.14
grid-carbon-intensity,0.01,0.30,-0.29,0.00,0.60,0.35,1.00,1.00,-0.14
cfe-ratio,0.02,0.07,-0.19,0.00,-0.79,-0.45,-0.14,-0.14,1.00


Initial conclusions from above :

- there is little/no difference in correlation if we exclude rows containing empty PUE values or not
- There seems to be a high correlation between "grid carbon intensity average consumption annual" and "provider carbon intensity average consumption hourly"
- There seems to be no correlation between "provider-carbon-intensity-market-annual" and anything else, suggesting it has empty values
- there is a strong inverse relationship between "Carbon Free Energy ratio" and "provider-carbon-intensity-average-consumption-hourly", which makes sense as the cfe-ratio is a measurement of how much carbon producing, vs carbon free energy a provider is consuming
- There is a strong correlation between grid-carbon-intensity and provider-carbon-intensity-average-consumption-hourly : this is suggestive that in areas where there is a high amount of carbon based energy, providers will consume a higher amount

### Investigating correlation between the ratio of CFE (Carbon Free Energy) to CEE (Carbon Emitting Energy) between Cloud providers and the Energy supplier for their region

In [155]:
import plotly.express as px
all_year_df = all_year_df.rename(columns={'provider-carbon-intensity-average-consumption-hourly':'provider carbon intensity',
                                             'grid-carbon-intensity-average-consumption-annual': 'grid carbon intensity'})
scatter_graph = px.scatter(all_year_df,
    x='provider carbon intensity',
    y='grid carbon intensity',
    trendline='ols')
scatter_graph.show()

Using the linear regression model, we can extrapolate on the correlation between provider and grid carbon intensity (identified in our heatmap above) and demonstrate a positive linear relationship between the two variables.   There are a few outliers but the general trend is clear.

## Linear regression by cloud provider
If we take the above to the next level of detail we can see if all cloud providers follow that trend:


In [156]:
scatter_graph = px.scatter(all_year_df,
    x='provider carbon intensity',
    y='grid carbon intensity',
    color='cloud-provider',
    symbol='cloud-provider',
    trendline='ols')
scatter_graph.show()

So the results of this are unexpected - there seems to be no data for cloud providers other than 'Google Cloud' on our linear regression graph.

To check why -

In [157]:

'''Create a summary table showing how many total rows there are for each cloud provider'''
all_rows_regardless_of_carbon = all_year_df['cloud-provider'].value_counts().to_frame().rename(columns={'count':'total rows'})
'''Create a summary table showing how many rows there are for each cloud provider when grid carbon intensity is not empty'''
not_empty_grid_carbon = all_year_df[all_year_df['grid carbon intensity'].notna()]['cloud-provider'].value_counts().to_frame().rename(columns={'count':'grid carbon intensity'})
'''Create a summary table showing how many rows there are for each cloud provider when provider carbon intensity is not empty'''
not_empty_provider_carbon = all_year_df[all_year_df['provider carbon intensity'].notna()]['cloud-provider'].value_counts().to_frame().rename(columns={'count':'provider carbon intensity'})
'''Merge summary tables together by joining on cloud provider name.  As we've renamed the count column, they do not conflict when merging'''
combined_table = pd.merge(all_rows_regardless_of_carbon, not_empty_grid_carbon, how='left', on='cloud-provider')
combined_table = pd.merge(combined_table, not_empty_provider_carbon, how='left', on='cloud-provider')
'''Use the function i wrote earlier to convert any floats to int and fill in any gaps with zero'''
combined_table = cast_float_to_int_and_set_nan_to_zero(combined_table)
'''display table - i'll add some styling later'''
display(HTML('<span style="color: white;font-style:bold"><br><h2>Comparing how many entries cloud-providers have in the dataset for grid and provider carbon intensity</h2></span>'))
display(combined_table)


,total rows,grid carbon intensity,provider carbon intensity
cloud-provider,,,
Google Cloud,238,163,238
Microsoft Azure,70,22,0
Amazon Web Services,70,30,0


Unfortunately we can see from above Amazon or Microsoft have provided no data about their carbon intensity, and only 31 - 42% of the data they have provided has details about the grid carbon intensity.

Google Cloud is more complete, with 100% coverage for provider carbon intensity, but only 68% coverage for grid carbon intensity.

Using data from Google alone is not ideal, but it does provide us some insight into the linear relationship between grid and carbon intensity.

## Provider Carbon Intensity by Region

Accepting that our analysis may not be an accurate reflection of the CFE / CEE ratio for Azure and AWS, we an still use the data to make an attempt to identify correlation between regions and carbon intensity.

First we select only the rows with data for both grid and provider;

In [158]:
'''make a copy of only the rows that have data in both columns'''
grid_and_provider_df = all_year_df[all_year_df['grid carbon intensity'].notna() & all_year_df['provider carbon intensity'].notna()].copy()
'''check the number of rows in the filtered dataset'''
grid_and_provider_df.shape[0]

163

### Check what regions we have



In [159]:
unique_locations = grid_and_provider_df[['cloud-region','cfe-region','em-zone-id','location']].sort_values(by=['cloud-region','cfe-region','em-zone-id']).drop_duplicates()
'''Show a sample of the data'''
display(HTML('<span style="color: white;font-style:bold"><br><h2>Sample of unique locations</h2></span>'))
display(unique_locations.sample(10))
regional_counts = dict()
for location_type in list(unique_locations.columns):
    regional_counts[location_type] = unique_locations[location_type].value_counts().to_frame().shape[0]

display(HTML('<span style="color: white;font-style:bold"><br><h2>Tally of how many unique entries for each location type there are</h2></span>'))
display(pd.DataFrame(pd.Series(regional_counts).to_frame().rename(columns={0:'unique entries'})))


,cloud-region,cfe-region,em-zone-id,location
33,us-east5,PJM,US-MIDA-PJM,Columbus
0,africa-south1,South Africa,ZA,Johannesburg
6,asia-south1,Maharashtra,IN-WE,Mumbai
3,asia-northeast1,Tokyo,JP-TK,Tokyo
38,us-west4,NVE,US-NW-NEVP,Las Vegas
273,europe-west2,Germany,DE,London
12,europe-central2,Poland,PL,Warsaw
10,australia-southeast1,New South Wales,AU-NSW,Sydney
21,europe-west9,France,FR,Paris
8,asia-southeast1,Singapore,SG,Singapore


,unique entries
cloud-region,38
cfe-region,36
em-zone-id,36
location,41


Reviewing the above there doesn't seem to be much grouping around the location data - the expectation was that we'd see location be part of much smaller groups denoted by em-zone-id but that doesn't seem to be the case.

Still we can make an attempt at showing a relationship between location and carbon intensity:

## Relationship between Cloud Location and Carbon Intensity

In [160]:
'''Attempt to group the locations by grid carbon intensity, by getting the average value : selecting cfe-region as it is the most readable of the two smaller location groups'''
grid_grouped_df = grid_and_provider_df.groupby('cfe-region', as_index=False)['grid carbon intensity'].mean()
grid_grouped_df = cast_float_to_int_and_set_nan_to_zero(grid_grouped_df)
'''Now do the same for provider'''
provider_grouped_df = grid_and_provider_df.groupby('cfe-region', as_index=False)['provider carbon intensity'].mean()
provider_grouped_df = cast_float_to_int_and_set_nan_to_zero(provider_grouped_df)
combined_table = pd.merge(grid_grouped_df, provider_grouped_df, how='left', on='cfe-region').sort_values(by=['grid carbon intensity'])
display(combined_table.sample(5))
combined_table.rename(columns={'provider carbon intensity':'provider','grid carbon intensity':'grid'}, inplace=True)
combined_table = combined_table.melt(id_vars='cfe-region', value_vars=['grid', 'provider'], var_name='carbon intensity type', value_name='Co2g Kwh')

line_graph = px.line(combined_table, x='cfe-region', y='Co2g Kwh', color='carbon intensity type', title='Cloud provider location vs grid carbon intensity')
line_graph.show()


,cfe-region,grid carbon intensity,provider carbon intensity
16,NVE,493,416
25,SC,629,501
6,Finland,128,95
9,Great Britain,214,188
3,Central Brazil,122,92


From the above graph we can see that Poland, Maharashtra and South Africa have the highest grid & provider carbon intensity values, with Quebec, Ontario and France having the lowest.

It would seem that Canada would be a good place to locate Cloud Compute workloads, at least on GCP, to reduce the generation of Scope 3 emissions via non Carbon Free energy usage to power them.

## Profiling Report Generator for reference

In [161]:
'''This section is for generating a Profiling report so we can have some insight into the data
    For the moment it is commented out as we have a pre-generated version'''
#%pip install ydata-profiling
#from ydata_profiling import ProfileReport
#profile = ProfileReport(df, title="Cloud Computing CFE 2019-2023_cleansed")
#profile.to_file("cloud_compute_cfe_2019-2013_cleansed.html")

'This section is for generating a Profiling report so we can have some insight into the data\n    For the moment it is commented out as we have a pre-generated version'